In [1]:
import os
import random

import cv2
import numpy as np
import scipy.io as scio
import torch
import torch.nn as nn
from PIL import Image
from luo import TicToc
from skimage import transform
from skimage import transform as TR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
# !gdown --id 1eAymcrGvjlnGt3amBxXPvmKJODtE34ZD

In [3]:
# !unzip data.zip

### Utils

In [4]:
def box_center(points):
    """
        support two input ways
        4 points: x1, y1, x2, y2, x3, y3, x4, y4
        2 points: lt_x1, lt_y1, rd_x2, rd_y2
    """
    if len(points) == 4:
        x1, y1, x2, y2 = points
        x3, y3, x4, y4 = x2, y1, x1, y2
    elif len(points) == 8:
        x1, y1, x2, y2, x3, y3, x4, y4 = points
    else:
        raise ("please input 2 points or 4 points, check it")
    center_x = round((x1 + x2 + x3 + x4) / 4)
    center_y = round((y1 + y2 + y3 + y4) / 4)
    return center_x, center_y, x1, y1, x3, y3, x2, y2, x4, y4


def triangle_center(points):
    if len(points) == 6:
        x1, y1, x2, y2, x3, y3 = points
    else:
        raise ("please input 3 points, check it")
    center_x = round((x1 + x2 + x3) / 3)
    center_y = round((y1 + y2 + y3) / 3)
    return center_x, center_y


def sorted_boxes(boxes):
    # sorted by the left top point's x location
    boxes = sorted(boxes, key=lambda box: box[0])
    return boxes


def create_affine_boxes(boxes):
    affine_boxes = []
    if len(boxes) == 1:
        return affine_boxes
    for boxes_1, boxes_2 in zip(boxes[:-1], boxes[1:]):
        center_x1, center_y1, x1, y1, x3, y3, x2, y2, x4, y4 = box_center(boxes_1)
        points_x1, points_y1 = triangle_center([center_x1, center_y1, x1, y1, x2, y2])
        points_x2, points_y2 = triangle_center([center_x1, center_y1, x3, y3, x4, y4])
        center_x2, center_y2, x1, y1, x3, y3, x2, y2, x4, y4 = box_center(boxes_2)
        points_x3, points_y3 = triangle_center([center_x2, center_y2, x1, y1, x2, y2])
        points_x4, points_y4 = triangle_center([center_x2, center_y2, x3, y3, x4, y4])
        affine_boxes.append([points_x1, points_y1, points_x3, points_y3, points_x4, points_y4, points_x2, points_y2, ])
    return affine_boxes


def find_min_rectangle(points):
    if len(points) == 4:
        x1, y1, x2, y2 = points
        x3, y3, x4, y4 = x2, y1, x1, y2
    elif len(points) == 8:
        x1, y1, x2, y2, x3, y3, x4, y4 = points
    else:
        raise ("please input 2 points or 4 points, check it")
    lt_x = min(x1, x2, x3, x4)
    lt_y = min(y1, y2, y3, y4)
    rd_x = max(x1, x2, x3, x4)
    rd_y = max(y1, y2, y3, y4)
    return np.float32([[lt_x, lt_y], [rd_x, lt_y], [rd_x, rd_y], [lt_x, rd_y]]), int(rd_x - lt_x), int(rd_y - lt_y)


def gaussian_kernel_2d_opencv(kernel_size=(3, 3)):
    ky = cv2.getGaussianKernel(kernel_size[0], int(kernel_size[0] / 4))
    kx = cv2.getGaussianKernel(kernel_size[1], int(kernel_size[1] / 4))
    return np.multiply(ky, np.transpose(kx))


def aff_gaussian(gaussian, box, pts, deta_x, deta_y):
    de_x, de_y = box[0]
    box = box - [de_x, de_y]
    pts = pts - [de_x, de_y]
    M = cv2.getPerspectiveTransform(box, pts)
    res = cv2.warpPerspective(gaussian, M, (deta_y, deta_x))
    return res


def rotate(angle, image):
    h, w = image.shape[1:]
    image = image.transpose((1, 2, 0))

    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    image = cv2.warpAffine(image, M, (w, h))
    image = image.transpose((2, 0, 1))

    return image, M


def rotate_point(M, x, y):
    point = np.array([x, y, 1])
    x, y = M.dot(point)
    return x, y


In [5]:
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, char_gt, aff_gt = sample["image"], sample["char_gt"], sample["aff_gt"]
        h, w = image.shape[1:]
        new_h, new_w = self.output_size
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[:, top:top + new_h, left:left + new_w]
        char_gt = char_gt[top:top + new_h, left:left + new_w]
        aff_gt = aff_gt[top:top + new_h, left:left + new_w]

        sample = {'image': image, 'char_gt': char_gt, 'aff_gt': aff_gt}
        return sample


class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, char_gt, aff_gt = sample["image"], sample["char_gt"], sample["aff_gt"]
        h, w = image.shape[1:]
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        image = image.transpose((1, 2, 0))
        image = transform.resize(image, (new_h, new_w))
        char_gt = transform.resize(char_gt, (new_h, new_w))
        aff_gt = transform.resize(aff_gt, (new_h, new_w))
        image = image.transpose((2, 0, 1))
        sample = {'image': image, 'char_gt': char_gt, 'aff_gt': aff_gt}
        return sample


class RedomRescale(object):
    def __init__(self, output_size_list):
        self.output_size_list

    def __call__(self, sample):
        length = len(self.output_size_list)
        idx = random.randint(0, length - 1)

        return sample


class Random_change(object):
    def __init__(self, random_bright, random_swap, random_contrast, random_saturation, random_hue):
        self.random_bright = random_bright
        self.random_swap = random_swap
        self.random_contrast = random_contrast
        self.random_saturation = random_saturation
        self.random_hue = random_hue

    def __call__(self, sample):
        image, char_gt, aff_gt = sample["image"], sample["char_gt"], sample["aff_gt"]
        if random.random() < self.random_bright:
            delta = random.uniform(-32, 32)
            image += delta
            image = image.clip(min=0, max=255)

        perms = ((0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0))
        if random.random() < self.random_swap:
            swap = perms[random.randrange(0, len(perms))]
            image = image[swap, :, :]

        if random.random() < self.random_contrast:
            alpha = random.uniform(0.5, 1.5)
            image *= alpha
            image = image.clip(min=0, max=255)

        if random.random() < self.random_saturation:
            image[1, :, :] *= random.uniform(0.5, 1.5)

        if random.random() < self.random_hue:
            image[0, :, :] += random.uniform(-18.0, 18.0)
            image[0, :, :][image[0, :, :] > 360.0] -= 360.0
            image[0, :, :][image[0, :, :] < 0.0] += 360.0

        sample = {'image': image, 'char_gt': char_gt, 'aff_gt': aff_gt}
        return sample


def random_resize_collate(batch):
    size = (320, 416, 480, 576, 640)
    random_size = size[random.randint(0, 4)]
    half_size = int(random_size / 2)
    images = []
    char_gts = []
    aff_gts = []
    for data in batch:
        images.append(data["image"])
        char_gts.append(data["char_gt"])
        aff_gts.append(data["aff_gt"])

    tr_images = []
    tr_char_gts = []
    tr_aff_gts = []
    for image, char_gt, aff_gt in zip(images, char_gts, aff_gts):
        image = image.transpose((1, 2, 0))
        image = transform.resize(image, (random_size, random_size))
        image = image.transpose((2, 0, 1))
        char_gt = transform.resize(char_gt, (half_size, half_size))
        aff_gt = transform.resize(aff_gt, (half_size, half_size))
        tr_images.append(torch.from_numpy(image))
        tr_char_gts.append(torch.from_numpy(char_gt))
        tr_aff_gts.append(torch.from_numpy(aff_gt))
    tr_images = torch.stack(tr_images, 0)
    tr_char_gts = torch.stack(tr_char_gts, 0)
    tr_aff_gts = torch.stack(tr_aff_gts, 0)
    sample = {
        'image': tr_images,
        'char_gt': tr_char_gts,
        'aff_gt': tr_aff_gts,
    }
    return sample

### Dataloader

In [6]:
class SynthText(Dataset):
    def __init__(self, data_dir_path=None, random_rote_rate=None, data_file_name=None, istrain=True,
                 image_size=(3, 640, 640), down_rate=2, transform=None):
        # check data path
        if data_dir_path is None:
            data_dir_path = "./data/SynthText"
        if data_file_name is None:
            data_file_name = "gt.mat"
        self.data_dir_path = data_dir_path
        self.data_file_name = data_file_name
        print("load data, please wait a moment...")

        self.agt = scio.loadmat(os.path.join(self.data_dir_path, self.data_file_name))
        self.istrain = istrain
        self.gt = {}
        if istrain:
            self.gt["txt"] = self.agt["txt"][0][:-1][:-100]
            self.gt["imnames"] = self.agt["imnames"][0][:-100]
            self.gt["charBB"] = self.agt["charBB"][0][:-100]
            self.gt["wordBB"] = self.agt["wordBB"][0][:-100]
        else:
            self.gt["txt"] = self.agt["txt"][0][-100:]
            self.gt["imnames"] = self.agt["imnames"][0][-100:]
            self.gt["charBB"] = self.agt["charBB"][0][-100:]
            self.gt["wordBB"] = self.agt["wordBB"][0][-100:]

        self.image_size = image_size
        self.down_rate = down_rate
        self.transform = transform
        self.random_rote_rate = random_rote_rate

    def __len__(self):
        return self.gt["txt"].shape[0]

    def resize(self, image, char_label, word_laebl):
        w, h = image.size
        img = np.zeros(self.image_size)
        rate = self.image_size[2] / self.image_size[1]
        rate_pic = w / h

        if rate_pic > rate:
            resize_h = int(self.image_size[2] / rate_pic)
            image = image.resize((self.image_size[2], resize_h), Image.ANTIALIAS)
            image = np.array(image)
            if self.image_size[0] == 3:
                if len(image.shape) == 2:
                    image = np.tile(image, (3, 1, 1))
                else:
                    image = image.transpose((2, 0, 1))

            img[:, :resize_h, :] = image
            char_label = char_label * (resize_h / h)
            word_laebl = word_laebl * (resize_h / h)
        else:
            resize_w = int(rate_pic * self.image_size[1])
            image = image.resize((resize_w, self.image_size[1]), Image.ANTIALIAS)
            image = np.array(image)
            if self.image_size[0] == 3:
                if len(image.shape) == 2:
                    image = np.tile(image, (3, 1, 1))
                else:
                    image = image.transpose((2, 0, 1))

            img[:, :, :resize_w] = np.array(image)
            char_label = char_label * (resize_w / w)
            word_laebl = word_laebl * (resize_w / w)
        return img, char_label, word_laebl

    def __getitem__(self, idx):
        img_name = self.gt["imnames"][idx]
        img_name = img_name.replace(" ", "")
        image = Image.open(os.path.join(self.data_dir_path, img_name))
        char_label = self.gt["charBB"][idx].transpose(2, 1, 0)
        if len(self.gt["wordBB"][idx].shape) == 3:
            word_laebl = self.gt["wordBB"][idx].transpose(2, 1, 0)
        else:
            word_laebl = self.gt["wordBB"][idx].transpose(1, 0)[np.newaxis, :]
        txt_label = self.gt["txt"][idx]

        img, char_label, word_laebl = self.resize(image, char_label, word_laebl)

        if self.random_rote_rate:
            angel = random.randint(0 - self.random_rote_rate, self.random_rote_rate)
            img, M = rotate(angel, img)

        char_gt = np.zeros((int(self.image_size[1]), int(self.image_size[2])))
        aff_gt = np.zeros((int(self.image_size[1]), int(self.image_size[2])))

        line_boxes = []
        char_index = 0
        word_index = 0
        for txt in txt_label:
            for strings in txt.split("\n"):
                for string in strings.split(" "):
                    if string == "":
                        continue
                    char_boxes = []
                    for char in string:
                        x0, y0 = char_label[char_index][0]
                        x1, y1 = char_label[char_index][1]
                        x2, y2 = char_label[char_index][2]
                        x3, y3 = char_label[char_index][3]

                        if self.random_rote_rate:
                            x0, y0 = rotate_point(M, x0, y0)
                            x1, y1 = rotate_point(M, x1, y1)
                            x2, y2 = rotate_point(M, x2, y2)
                            x3, y3 = rotate_point(M, x3, y3)

                        x0, y0, x1, y1, x2, y2, x3, y3 = int(round(x0)), int(round(y0)), int(round(x1)), int(
                            round(y1)), int(round(x2)), int(round(y2)), int(round(x3)), int(round(y3))
                        char_boxes.append([x0, y0, x1, y1, x2, y2, x3, y3])
                        box, deta_x, deta_y = find_min_rectangle([x0, y0, x1, y1, x2, y2, x3, y3])
                        if deta_x <= 0 or deta_x >= self.image_size[2] or deta_y <= 0 or deta_y >= self.image_size[1]:
                            # print(idx, deta_x, deta_y)
                            char_index += 1
                            continue
                        try:
                            gaussian = gaussian_kernel_2d_opencv(kernel_size=(deta_y, deta_x))
                            pts = np.float32([[x0, y0], [x1, y1], [x2, y2], [x3, y3]])
                            res = aff_gaussian(gaussian, box, pts, deta_y, deta_x)
                        except:
                            char_index += 1
                            continue

                        min_x = min(x0, x1, x2, x3)
                        min_y = min(y0, y1, y2, y3)

                        if np.max(res) > 0:
                            mx = 1 / np.max(res)
                            res = mx * res
                            gh, gw = res.shape
                            for th in range(gh):
                                for tw in range(gw):
                                    if 0 < min_y + th < char_gt.shape[0] and 0 < min_x + tw < char_gt.shape[1]:
                                        try:
                                            char_gt[min_y + th, min_x + tw] = max(char_gt[min_y + th, min_x + tw],
                                                                                  res[th, tw])
                                        except:
                                            print(idx, min_y + th, min_x + tw)

                        char_index += 1
                    word_index += 1
                    line_boxes.append(char_boxes)
        affine_boxes = []
        for char_boxes in line_boxes:
            affine_boxes.extend(create_affine_boxes(char_boxes))
            for points in affine_boxes:
                x0, y0, x1, y1, x2, y2, x3, y3 = points[0], points[1], points[2], points[3], points[4], points[5], \
                                                 points[6], points[7]
                box, deta_x, deta_y = find_min_rectangle(points)
                if deta_x <= 0 or deta_x >= self.image_size[2] or deta_y <= 0 or deta_y >= self.image_size[1]:
                    continue
                try:
                    gaussian = gaussian_kernel_2d_opencv(kernel_size=(deta_y, deta_x))
                    pts = np.float32([[x0, y0], [x1, y1], [x2, y2], [x3, y3]])
                    res = aff_gaussian(gaussian, box, pts, deta_y, deta_x)
                except:
                    continue
                min_x = min(x0, x1, x2, x3)
                min_y = min(y0, y1, y2, y3)

                if np.max(res) > 0:
                    mx = 1 / np.max(res)
                    res = mx * res
                    gh, gw = res.shape
                    for th in range(gh):
                        for tw in range(gw):
                            if 0 < min_y + th < aff_gt.shape[0] and 0 < min_x + tw < aff_gt.shape[1]:
                                try:
                                    aff_gt[min_y + th, min_x + tw] = max(aff_gt[min_y + th, min_x + tw], res[th, tw])
                                except:
                                    print(idx, min_y + th, min_x + tw)
        sample = {
            'image': img,
            'char_gt': char_gt,
            'aff_gt': aff_gt,
            # 'affine_boxes': affine_boxes,
            # 'line_boxes': line_boxes,
            # 'char_label': char_label
        }

        if self.transform:
            sample = self.transform(sample)

        sample['char_gt'] = TR.resize(sample['char_gt'], (
            int(self.image_size[1] / self.down_rate), int(self.image_size[2] / self.down_rate)))
        sample['aff_gt'] = TR.resize(sample['aff_gt'], (
            int(self.image_size[1] / self.down_rate), int(self.image_size[2] / self.down_rate)))

        return sample


### Model

In [7]:
class Base_with_bn_block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, up=False):
        super(Base_with_bn_block, self).__init__()
        self.up = up
        if up:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear")
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=int(kernel_size / 2))
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self._initialize_weights()

    def forward(self, x):

        if self.up:
            x = self.up(x)
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


class Base_down_block(nn.Module):
    def __init__(self, in_channels, out_channels, times):
        super(Base_down_block, self).__init__()

        self.blocks = [Base_with_bn_block(in_channels, out_channels, 3)]
        for i in range(times - 1):
            self.blocks += [Base_with_bn_block(out_channels, out_channels, 3)]
        self.blocks = nn.Sequential(*self.blocks)

    def forward(self, x):
        out = self.blocks(x)
        return out


class Base_up_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Base_up_block, self).__init__()
        self.block1 = Base_with_bn_block(in_channels, out_channels * 2, 1, up=True)
        self.block2 = Base_with_bn_block(out_channels * 2, out_channels, 3)

    def forward(self, x1, x2):
        out = torch.cat([x1, x2], 1)
        out = self.block1(out)
        out = self.block2(out)
        return out


class UP_VGG(nn.Module):
    def __init__(self):
        super(UP_VGG, self).__init__()
        self.layers = nn.Sequential(*[Base_down_block(3, 64, 2),
                                      Base_down_block(64, 128, 2),
                                      Base_down_block(128, 256, 3),
                                      Base_down_block(256, 512, 3),
                                      Base_down_block(512, 512, 3),
                                      Base_down_block(512, 512, 3), ])

        self.up_layers = nn.Sequential(*[Base_up_block(512 + 512, 256),
                                         Base_up_block(512 + 256, 128),
                                         Base_up_block(256 + 128, 64),
                                         Base_up_block(128 + 64, 32)])

        self.detector = nn.Sequential(*[Base_with_bn_block(32, 32, 3),
                                        Base_with_bn_block(32, 32, 3),
                                        Base_with_bn_block(32, 16, 3),
                                        Base_with_bn_block(16, 16, 1)])

        self.region = nn.Conv2d(16, 1, 1)
        self.affinity = nn.Conv2d(16, 1, 1)

        self.pooling = nn.MaxPool2d(2, 2)

    def forward(self, x):
        features = []
        for i in range(5):
            x = self.layers[i](x)
            x = self.pooling(x)
            features.append(x)
        x = self.layers[-1](x)
        for index in range(4):
            x = self.up_layers[index](features[-index - 1], x)
        x = self.detector(x)
        reg = self.region(x)
        aff = self.affinity(x)
        return reg, aff


### Loss and optimizer

In [8]:
class MSE_OHEM_Loss(nn.Module):
    def __init__(self):
        super(MSE_OHEM_Loss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="none")

    def forward(self, output_imgs, target_imgs):
        loss_every_sample = []
        batch_size = output_imgs.size(0)
        for i in range(batch_size):
            output_img = output_imgs[i].view(1, -1)
            target_img = target_imgs[i].view(1, -1)
            positive_mask = (target_img > 0).float()
            sample_loss = self.mse_loss(output_img, target_img)

            positive_loss = torch.masked_select(sample_loss, positive_mask.byte())
            negative_loss = torch.masked_select(sample_loss, 1 - positive_mask.byte())
            num_positive = int(positive_mask.sum().data.cpu().item())

            k = num_positive * 3
            num_all = output_img.shape[1]
            if k + num_positive > num_all:
                k = int(num_all - num_positive)
            if k < 10:
                avg_sample_loss = sample_loss.mean()
            else:
                negative_loss_topk, _ = torch.topk(negative_loss, k)
                avg_sample_loss = positive_loss.mean() + negative_loss_topk.mean()
            loss_every_sample.append(avg_sample_loss)
        return torch.stack(loss_every_sample, 0).mean()



### Train

In [9]:
image_size = (3, 640, 640)
train_dataset = SynthText(image_size=image_size,
                          random_rote_rate=30,
                          transform=transforms.Compose([
                              RandomCrop((480, 480)),
                              Rescale((640, 640)),
                              Random_change(0.5, 0.5, 0.5, 0.5, 0.5)]))

train_dataLoader = DataLoader(train_dataset, 2, shuffle=True, num_workers=4,
                              collate_fn=random_resize_collate)
test_dataset = SynthText(image_size=image_size, istrain=False)

test_dataLoader = DataLoader(test_dataset, 4, shuffle=False, num_workers=4)

vgg = UP_VGG()
vgg = vgg.to("cuda")
print(TicToc.format_time(), "finish load")

optimizer = torch.optim.Adam(vgg.parameters(), lr=0.001)
# crite = nn.MSELoss(reduction="mean")
# l1_crite = nn.SmoothL1Loss(reduction="mean")
# cls_crite = nn.CrossEntropyLoss(reduction="mean")
loss_fn = MSE_OHEM_Loss()
loss_fn = loss_fn.to("cuda")
print(TicToc.format_time(), " training.........")

for e in range(1):
    # train
    total_loss = 0.0
    char_loss = 0.0
    aff_loss = 0.0
    b_total_loss = 0.0
    b_char_loss = 0.0
    b_aff_loss = 0.0
    vgg.train()

    for i, batch_data in enumerate(train_dataLoader):
        image = batch_data["image"].type(torch.FloatTensor) / 255 - 0.5
        image = image.to("cuda")
        reg, aff = vgg(image)
        predict_r = torch.squeeze(reg, dim=1)
        predict_l = torch.squeeze(aff, dim=1)

        targets_r = batch_data["char_gt"].type(torch.FloatTensor)
        targets_r = targets_r.to("cuda")

        targets_l = batch_data["aff_gt"].type(torch.FloatTensor)
        targets_l = targets_l.to("cuda")

        optimizer.zero_grad()
        loss_r = loss_fn(predict_r, targets_r)
        loss_l = loss_fn(predict_l, targets_l)

        loss = loss_r + loss_l  # + loss_cls
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        char_loss += loss_r.item()
        aff_loss += loss_l.item()
        b_total_loss += loss.item()
        b_char_loss += loss_r.item()
        b_aff_loss += loss_l.item()

        if i % 1000 == 999:
            print(TicToc.format_time(), e, i, b_total_loss, b_char_loss, b_aff_loss)
            b_total_loss = 0.0
            b_char_loss = 0.0
            b_aff_loss = 0.0

    print("Train ", TicToc.format_time(), e, total_loss, char_loss, aff_loss)

load data, please wait a moment...
load data, please wait a moment...
2021-05-17 23:59:26.264 finish load
2021-05-17 23:59:26.265  training.........


/home/chelvan/.local/lib/python3.8/site-packages/torch/nn/functional.py:3454: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
<ipython-input-8-b2c35ca98824>:15: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  positive_loss = torch.masked_select(sample_loss, positive_mask.byte())
<ipython-input-8-b2c35ca98824>:16: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  negative_loss = torch.masked_select(sample_loss, 1 - positive_mask.byte())


2021-05-18 00:04:19.799 0 999 218.6844322308898 133.71892590541393 84.96550659229979
337 186 171


KeyboardInterrupt: 